# Physics Simulator
Experimental Project by Sahil Dev

In [1]:
import pygame
import pygame_menu
import numpy as np

pygame 2.1.2 (SDL 2.0.18, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
pygame-menu 4.2.2


## Class Definitions

In [2]:
# class Vec1D(np.ndarray):
#     def __new__(cls, magnitude):
#         if isinstance(magnitude, float):
#             return np.asarray(magnitude).view(cls)
#         raise Exception('Vec2D must be 1 dimensional.')
    
class Vec2D(np.ndarray):
    def __new__(cls, iterable):
        if len(iterable) != 2:
            raise Exception('Vec2D must be 2 dimensional.')
        vec = np.asarray(iterable).view(cls)
        return vec
        
    def x(self):
        return self[0]
    
    def y(self):
        return self[1]
    
a, b = Vec2D([2, 1]), Vec2D([2, 4])
c = 2*a - b/2
c

Vec2D([3., 0.])

In [3]:
class Acceleration(Vec2D):
    pass

class Velocity(Vec2D):
    # need to have self as leftmost value to preserve type
    def update(self, timestep, acceleration):
        return self + timestep * acceleration

class Position(Vec2D):
    def distance(self, point):
        return np.linalg.norm((self - point))
    
    def xdistance(self, point):
        return abs(self.x() - point.x())
    
    def ydistance(self, point):
        return abs(self.y() - point.y())
    
    # need to have self as leftmost value to preserve type
    def update(self, timestep, velocity, acceleration=Acceleration([0, 0])):
        return self + acceleration/2 * (timestep**2) + velocity * timestep

acc = Acceleration((0, -9.807))
vel = Velocity((0, 0))
pos = Position((0, 380))

# it takes just under 9 seconds to fall from the empire state building
for i in range(4):
    print(f't={3 * i}\tpos={pos}\t vel={vel}')
    pos = pos.update(3, vel, acc)
    vel = vel.update(3, acc)

t=0	pos=[  0 380]	 vel=[0 0]
t=3	pos=[  0.     335.8685]	 vel=[  0.    -29.421]
t=6	pos=[  0.    203.474]	 vel=[  0.    -58.842]
t=9	pos=[  0.     -17.1835]	 vel=[  0.    -88.263]


In [4]:
# represents any object within the game environment
class GameObject:
    def __init__(self):
        pass
    
    def update(self, timestep):
        raise NotImplementedError('Must override abstract method.')
    
    def checkCollision(self, gameobject):
        raise NotImplementedError('Must override abstract method.')

    def onCollision(self, gameobject, data=None):
        raise NotImplementedError('Must override abstract method.')
        
class Viewport(GameObject):
    def fromPix(screendim, dim):
        width, height = screendim
        widthmeters, heightmeters = dim
        
        if width is None or height is None:
            raise Exception('Must input screen dimensions.')
        if widthmeters is None and heightmeters is None:
            raise Exception('Must have at least one dimension defined.')
        
        aspectratio = width/height
        if widthmeters is None:
            widthmeters = heightmeters * aspectratio
        if heightmeters is None:
            heightmeters = widthmeters / aspectratio
        
        low = Position((-widthmeters/2, 0))
        high = Position((widthmeters/2, heightmeters))
        
        return Viewport(low, high)
        
    def __init__(self, low: Position, high: Position):
        super().__init__()
        self.low = low
        self.high = high
    
    # might need to change if viewport is allowed to change
    def update(self, timestep):
        pass
    
    # use other gameobject collision function
    def checkCollision(self, gameobject):
        if isinstance(gameobject, Viewport):
            return False
            # raise Exception('Attempted collision check with two Viewports.')
        return gameobject.checkCollision(self)
    
    # don't do anything to the viewport when colliding with an object
    def onCollision(self, gameobject, data=None):
        pass

In [5]:
# constants

GRAVITY = Acceleration((0, -9.807))
ORIGIN = Position((0, 0))
NO_VELOCITY = Velocity((0, 0))
NO_ACCELERATION = Acceleration((0, 0))

# x = -10m to +10m, y = 0m to +20m
DEFAULT_VIEWPORT = Viewport(Position((-10, 0)), Position((10, 20)))

In [6]:
# represents an object that responds to physics
class PhysicsObject(GameObject):
    def __init__(self, mass, position, velocity=NO_VELOCITY, gravity=GRAVITY, bounciness=1):
        super().__init__()
        self.mass = mass
        self.position = position
        self.velocity = velocity
        self.gravity = gravity
        self.bounciness = bounciness

    def update(self, timestep):
        self.position = self.position.update(timestep, self.velocity, self.gravity)
        self.velocity = self.velocity.update(timestep, self.gravity)
        self.gravity = self.gravity # no-op
        
class Circle(PhysicsObject):
    def __init__(self, mass, center, radius, velocity=NO_VELOCITY, gravity=GRAVITY, bounciness=1):
        super().__init__(mass, center, velocity=velocity, gravity=gravity, bounciness=bounciness)
        self.radius = radius
    
    def checkCollision(self, gameobject):
        collision, data = None, None
        
        # TODO: incorporate mass
        if isinstance(gameobject, Viewport):
            vport = gameobject
            highx_collision = (vport.high[0] - self.position[0]) < self.radius
            if highx_collision:
                data = 0
            highy_collision = (vport.high[1] - self.position[1]) < self.radius
            if highy_collision:
                data = 1
            lowx_collision = (self.position[0] - vport.low[0]) < self.radius
            if lowx_collision:
                data = 2
            lowy_collision = (self.position[1] - vport.low[1]) < self.radius
            if lowy_collision:
                data = 3
            
            collision = (lowx_collision or lowy_collision or highx_collision or highy_collision)
            
            return collision, data
        
        # TODO: return collision data
        elif isinstance(gameobject, Circle):
            dist = self.position.distance(gameobject.position)
            collision = (dist < self.radius + gameobject.radius)
            
            return collision, data
        
        else:
            pass
    
    def onCollision(self, gameobject, data=None):
        if isinstance(gameobject, Viewport):
            self.position = np.minimum(self.position, gameobject.high - self.radius)
            self.position = np.maximum(self.position, gameobject.low + self.radius)
            if data == 0: # right wall
                self.velocity = Velocity((-self.bounciness * abs(self.velocity[0]), self.velocity[1]))
            elif data == 1: # ceiling
                self.velocity = Velocity((self.velocity[0], -self.bounciness * abs(self.velocity[1])))
            elif data == 2: # left wall
                self.velocity = Velocity((self.bounciness * abs(self.velocity[0]), self.velocity[1]))
            elif data == 3: # floor
                self.velocity = Velocity((self.velocity[0], self.bounciness * abs(self.velocity[1])))
            else:
                pass
            
        if isinstance(gameobject, Circle):
            pass

## Game Logic

In [7]:
class Game:
    def __init__(self, viewport, objects, screendim):
        self.screendim = screendim
        self.viewport = viewport
        self.objects = objects
        
    def update(self, timestep):
        self.viewport.update(timestep) # probably won't ever do anything
        
        # run update step on all objects
        for gameobject in self.objects:
            gameobject.update(timestep)
        
        # run collision check against viewport
        for gameobject in self.objects:
            collision, data = gameobject.checkCollision(self.viewport)
            if collision:
                print(f'Detected collision between {self.viewport} and {type(gameobject)} at {gameobject.position}.')
                gameobject.onCollision(self.viewport, data)
        
        # run collision check on all object pairs
        for index, gameobject1 in enumerate(self.objects):
            for gameobject2 in self.objects[index:]:
            
                # check only one direction, should be symmetric (we'll see about that lol)
                collision, data = gameobject1.checkCollision(gameobject2)
                if collision:
                # print(f'Detected collision between {gameobject1} and {gameobject2}.')
                    
                    # run collision method for each object in collision
                    gameobject1.onCollision(gameobject2, data)
                    gameobject2.onCollision(gameobject1, data)
                    
    def pixFromVec(self, vec: Vec2D, centered=False):
        width, height = self.screendim # screen dimensions, in pixels
        low, high = self.viewport.low, self.viewport.high # viewport positions, in meters
        widthmeters, heightmeters = high - low # viewport dimensions, in meters
        
        if centered:
            # find distance from top left corner instead
            distfromleft = vec[0] - low[0]
            distfromtop = high[1] - vec[1]
            vec = Vec2D((distfromleft, distfromtop))
        
        # convert distance to pixels
        scale = Vec2D((width / widthmeters, height / heightmeters))
        return vec * scale
    
    def draw(self, window):
        for gameobject in self.objects:
            if isinstance(gameobject, Circle):
                center = self.pixFromVec(gameobject.position, centered=True)
                radius = self.pixFromVec(Vec2D((gameobject.radius, gameobject.radius)), centered=False)
                left, top = center - radius
                w, h = radius * 2
                rect = pygame.Rect(left, top, w, h)
                circle = pygame.draw.ellipse(window, (45, 40, 50), rect)
            
            else:
                pass

screendim = (1920, 1200)
vport = Viewport.fromPix(screendim, (10, None))
# vport = Viewport(Position((0, 0)), Position((1920, 1200)))
game = Game(vport, [], screendim)

# origin is at center bottom of screen
game.pixFromVec(Vec2D((0, 0)), centered=True)

Vec2D([ 960., 1200.])

In [8]:
game = Game(vport, [Circle(10, Position((1, 3)), 1), Circle(10, Position((-2, 2)), 1)], screendim)
for gameobject in game.objects:
    print(f'{type(gameobject)}: position={gameobject.position}, velocity={gameobject.velocity}')

<class '__main__.Circle'>: position=[1 3], velocity=[0 0]
<class '__main__.Circle'>: position=[-2  2], velocity=[0 0]


## Pygame GUI

In [9]:
def start(window, clock=pygame.time.Clock(), fps=60):
    screendim = window.get_size()
    
    # BASIC SETUP
    viewport = Viewport.fromPix(screendim, (20, None))
    game = Game(viewport,
                [Circle(10, Position((-3, 4)), 1, bounciness=0.97, velocity=Velocity((3, -2))),
                 Circle(10, Position((2, 6)), 1, bounciness=0.9, velocity=Velocity((-6, 6))),
                 Circle(10, Position((8, 3)), 0.5, bounciness=0.8, velocity=Velocity((-15, 15))),
                 Circle(10, Position((-6, 10)), 0.7, bounciness=0.85, velocity=Velocity((15, -15))),
                 Circle(10, Position((8, 10)), 0.3, bounciness=0.9, velocity=Velocity((10, 30)))
                ], screendim)
    
    running = True
    while running:
        # draw current frame
        window.fill((215, 210, 200))
        game.draw(window)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.KEYDOWN:
                if event.key in [pygame.K_q, pygame.K_ESCAPE]:
                    running = False
        
        # update
        pygame.display.update()
        clock.tick(fps)
        game.update(1/fps)

In [10]:
def main_menu(window):
    
    def start_game():
        clock = pygame.time.Clock()
        FPS = 60
        start(window, clock=clock, fps=FPS)
    
    menu = pygame_menu.Menu('Physics Simulator', 400, 400,
                            theme=pygame_menu.themes.THEME_DARK)
    menu.add.button('Play', start_game)
    menu.add.button('Quit', pygame_menu.events.EXIT)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            return

    menu.mainloop(window)

    # menu.add.button('Level Select', load_peg_file_menu, False)
    # menu.add.button('Create', create_level, [])
    # menu.add.button('Full Screen', full_screen_toggle)

In [11]:
def main():
    # pygame initialization
    print('Starting pygame window...')
    pygame.init()
    window = pygame.display.set_mode((0, 0), pygame.FULLSCREEN) # window size 0,0 for full screen
    pygame.display.set_caption("Physics Simulator")

    print('Starting main menu...')
    main_menu(window)

In [ ]:
main()

Starting pygame window...
Starting main menu...
Detected collision between <__main__.Viewport object at 0x0000021BFC917AC0> and <class '__main__.Circle'> at [ 8.83333333 12.46594792].
Detected collision between <__main__.Viewport object at 0x0000021BFC917AC0> and <class '__main__.Circle'> at [9.83333333 9.5245175 ].
Detected collision between <__main__.Viewport object at 0x0000021BFC917AC0> and <class '__main__.Circle'> at [ 6.7        -0.10204083].
Detected collision between <__main__.Viewport object at 0x0000021BFC917AC0> and <class '__main__.Circle'> at [2.         0.60522667].
Detected collision between <__main__.Viewport object at 0x0000021BFC917AC0> and <class '__main__.Circle'> at [-1.15        0.90197458].
Detected collision between <__main__.Viewport object at 0x0000021BFC917AC0> and <class '__main__.Circle'> at [-4.5        12.09479167].
Detected collision between <__main__.Viewport object at 0x0000021BFC917AC0> and <class '__main__.Circle'> at [ 2.35       12.42815404].
Dete

Detected collision between <__main__.Viewport object at 0x0000021BFC917AC0> and <class '__main__.Circle'> at [4.6323     0.96476378].
Detected collision between <__main__.Viewport object at 0x0000021BFC917AC0> and <class '__main__.Circle'> at [-0.92882     0.21349695].
Detected collision between <__main__.Viewport object at 0x0000021BFC917AC0> and <class '__main__.Circle'> at [0.74      0.4649681].
Detected collision between <__main__.Viewport object at 0x0000021BFC917AC0> and <class '__main__.Circle'> at [-2.60188    0.9739703].
Detected collision between <__main__.Viewport object at 0x0000021BFC917AC0> and <class '__main__.Circle'> at [3.6072     0.49420772].
Detected collision between <__main__.Viewport object at 0x0000021BFC917AC0> and <class '__main__.Circle'> at [9.0063     1.93123302].
Detected collision between <__main__.Viewport object at 0x0000021BFC917AC0> and <class '__main__.Circle'> at [0.31731265 0.63979125].
Detected collision between <__main__.Viewport object at 0x0000

Detected collision between <__main__.Viewport object at 0x0000021BFC917AC0> and <class '__main__.Circle'> at [-4.26256     0.49624023].
Detected collision between <__main__.Viewport object at 0x0000021BFC917AC0> and <class '__main__.Circle'> at [-4.59024    0.4961974].
Detected collision between <__main__.Viewport object at 0x0000021BFC917AC0> and <class '__main__.Circle'> at [-8.055216    0.96462979].
Detected collision between <__main__.Viewport object at 0x0000021BFC917AC0> and <class '__main__.Circle'> at [-4.91792     0.49623166].
Detected collision between <__main__.Viewport object at 0x0000021BFC917AC0> and <class '__main__.Circle'> at [-5.2456      0.49620425].
Detected collision between <__main__.Viewport object at 0x0000021BFC917AC0> and <class '__main__.Circle'> at [-5.57328     0.49622618].
Detected collision between <__main__.Viewport object at 0x0000021BFC917AC0> and <class '__main__.Circle'> at [-5.90096     0.49620864].
Detected collision between <__main__.Viewport obje

Detected collision between <__main__.Viewport object at 0x0000021BFC917AC0> and <class '__main__.Circle'> at [-2.2123968   0.49621644].
Detected collision between <__main__.Viewport object at 0x0000021BFC917AC0> and <class '__main__.Circle'> at [-9.7507406  3.1001505].
Detected collision between <__main__.Viewport object at 0x0000021BFC917AC0> and <class '__main__.Circle'> at [2.21272468 0.68578021].
Detected collision between <__main__.Viewport object at 0x0000021BFC917AC0> and <class '__main__.Circle'> at [4.58127   0.9731529].
Detected collision between <__main__.Viewport object at 0x0000021BFC917AC0> and <class '__main__.Circle'> at [-1.9502528   0.49621643].
Detected collision between <__main__.Viewport object at 0x0000021BFC917AC0> and <class '__main__.Circle'> at [-1.6881088   0.49621644].
Detected collision between <__main__.Viewport object at 0x0000021BFC917AC0> and <class '__main__.Circle'> at [-1.4259648   0.49621643].
Detected collision between <__main__.Viewport object at 

Detected collision between <__main__.Viewport object at 0x0000021BFC917AC0> and <class '__main__.Circle'> at [6.7004992  0.49621644].
Detected collision between <__main__.Viewport object at 0x0000021BFC917AC0> and <class '__main__.Circle'> at [5.6520331  0.68835419].
Detected collision between <__main__.Viewport object at 0x0000021BFC917AC0> and <class '__main__.Circle'> at [2.64006002 0.21897529].
Detected collision between <__main__.Viewport object at 0x0000021BFC917AC0> and <class '__main__.Circle'> at [6.9626432  0.49621644].
Detected collision between <__main__.Viewport object at 0x0000021BFC917AC0> and <class '__main__.Circle'> at [3.7387341  0.98166037].
Detected collision between <__main__.Viewport object at 0x0000021BFC917AC0> and <class '__main__.Circle'> at [7.2247872  0.49621644].
Detected collision between <__main__.Viewport object at 0x0000021BFC917AC0> and <class '__main__.Circle'> at [5.07299073 0.68946769].
Detected collision between <__main__.Viewport object at 0x0000

...

Detected collision between <__main__.Viewport object at 0x0000021BFC917340> and <class '__main__.Circle'> at [-9.24076245  0.29283114].
Detected collision between <__main__.Viewport object at 0x0000021BFC917340> and <class '__main__.Circle'> at [7.01768664 0.68895608].
Detected collision between <__main__.Viewport object at 0x0000021BFC917340> and <class '__main__.Circle'> at [7.40411277 0.979282  ].
Detected collision between <__main__.Viewport object at 0x0000021BFC917340> and <class '__main__.Circle'> at [0.77541291 0.49621644].
Detected collision between <__main__.Viewport object at 0x0000021BFC917340> and <class '__main__.Circle'> at [0.79342731 0.49621644].
Detected collision between <__main__.Viewport object at 0x0000021BFC917340> and <class '__main__.Circle'> at [-9.17011052  0.29283114].
Detected collision between <__main__.Viewport object at 0x0000021BFC917340> and <class '__main__.Circle'> at [7.06826844 0.68895608].
Detected collision between <__main__.Viewport object at 0x

Detected collision between <__main__.Viewport object at 0x0000021BFC917340> and <class '__main__.Circle'> at [1.22577287 0.49621644].
Detected collision between <__main__.Viewport object at 0x0000021BFC917340> and <class '__main__.Circle'> at [-8.32228736  0.29283114].
Detected collision between <__main__.Viewport object at 0x0000021BFC917340> and <class '__main__.Circle'> at [7.67524996 0.68895608].
Detected collision between <__main__.Viewport object at 0x0000021BFC917340> and <class '__main__.Circle'> at [1.24378727 0.49621644].
Detected collision between <__main__.Viewport object at 0x0000021BFC917340> and <class '__main__.Circle'> at [6.52540564 0.979282  ].
Detected collision between <__main__.Viewport object at 0x0000021BFC917340> and <class '__main__.Circle'> at [1.26180167 0.49621644].
Detected collision between <__main__.Viewport object at 0x0000021BFC917340> and <class '__main__.Circle'> at [-8.25163543  0.29283114].
Detected collision between <__main__.Viewport object at 0x

Detected collision between <__main__.Viewport object at 0x0000021BFC917340> and <class '__main__.Circle'> at [1.73017603 0.49621644].
Detected collision between <__main__.Viewport object at 0x0000021BFC917340> and <class '__main__.Circle'> at [-7.33316033  0.29283114].
Detected collision between <__main__.Viewport object at 0x0000021BFC917340> and <class '__main__.Circle'> at [8.38339507 0.68895608].
Detected collision between <__main__.Viewport object at 0x0000021BFC917340> and <class '__main__.Circle'> at [5.6466985 0.979282 ].
Detected collision between <__main__.Viewport object at 0x0000021BFC917340> and <class '__main__.Circle'> at [1.74819043 0.49621644].
Detected collision between <__main__.Viewport object at 0x0000021BFC917340> and <class '__main__.Circle'> at [1.76620483 0.49621644].
Detected collision between <__main__.Viewport object at 0x0000021BFC917340> and <class '__main__.Circle'> at [-7.2625084   0.29283114].
Detected collision between <__main__.Viewport object at 0x00

Detected collision between <__main__.Viewport object at 0x0000021BFC917340> and <class '__main__.Circle'> at [2.23457919 0.49621644].
Detected collision between <__main__.Viewport object at 0x0000021BFC917340> and <class '__main__.Circle'> at [-6.3440333   0.29283114].
Detected collision between <__main__.Viewport object at 0x0000021BFC917340> and <class '__main__.Circle'> at [4.76799136 0.979282  ].
Detected collision between <__main__.Viewport object at 0x0000021BFC917340> and <class '__main__.Circle'> at [9.09154018 0.68895608].
Detected collision between <__main__.Viewport object at 0x0000021BFC917340> and <class '__main__.Circle'> at [6.83291117 0.94173459].
Detected collision between <__main__.Viewport object at 0x0000021BFC917340> and <class '__main__.Circle'> at [2.25259359 0.49621644].
Detected collision between <__main__.Viewport object at 0x0000021BFC917340> and <class '__main__.Circle'> at [2.27060799 0.49621644].
Detected collision between <__main__.Viewport object at 0x00

Detected collision between <__main__.Viewport object at 0x0000021BFC917130> and <class '__main__.Circle'> at [-7.46        0.38531833].
Detected collision between <__main__.Viewport object at 0x0000021BFC917130> and <class '__main__.Circle'> at [9.74      6.5545263].
Detected collision between <__main__.Viewport object at 0x0000021BFC917130> and <class '__main__.Circle'> at [-9.54        2.51553192].
Detected collision between <__main__.Viewport object at 0x0000021BFC917130> and <class '__main__.Circle'> at [9.09     5.260705].
Detected collision between <__main__.Viewport object at 0x0000021BFC917130> and <class '__main__.Circle'> at [0.088125   0.48367186].
Detected collision between <__main__.Viewport object at 0x0000021BFC917130> and <class '__main__.Circle'> at [ 5.6905     12.20347352].
Detected collision between <__main__.Viewport object at 0x0000021BFC917130> and <class '__main__.Circle'> at [5.217      0.94739714].
Detected collision between <__main__.Viewport object at 0x0000